In [ ]:
from webbot import Browser
from bs4 import BeautifulSoup
import re
import pandas as pd

web = Browser(showWindow=False)
web.go_to('https://scele.cs.ui.ac.id/login/index.php') 
web.click('Sign in')
web.type(user_id , into='Email') #ganti dgn user id
web.click('NEXT' , tag='span')
web.type(password , into='Password' , id='passwordFieldId') #ganti password
web.click('Log in' , tag='span') 
web.go_to('https://scele.cs.ui.ac.id/user/index.php?id=3065&search&perpage=5000') #ganti dgn id course daa
source = web.get_page_source()
soup = BeautifulSoup(source)
links = soup.find_all("strong")
links = [link.a.get("href") for link in links]
user_id = [re.search("id=.*", user).group(0).strip("id=&") for user in links]
link_outline = ["https://scele.cs.ui.ac.id/report/outline/user.php?id="+uid+"&mode=outline" for uid in user_id]
web.go_to(link_outline[0])
outline = web.get_page_source()
outline = BeautifulSoup(outline)
forums = outline.find_all("a", attrs={"title":"Forum"})
forum_name = [forum.text for forum in forums]
col = ["Name"]+forum_name
df = pd.DataFrame(columns=col)
for link in link_outline:
    web.go_to(link)
    outline = web.get_page_source()
    outline = BeautifulSoup(outline)
    forums = outline.find_all("a", attrs={"title":"Forum"})
    df = df.append(pd.Series([outline.find("h2").text]+[forum.parent.parent.find_all("td", attrs={"valign" : "top"})[2].text for forum in forums], index=col), ignore_index=True)

df.to_excel("rekap.xlsx")